In [2]:
# Dependencies
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from matplotlib import pyplot as plt
import random
from yelpapi import YelpAPI
from pprint import pprint

# Census API Key
c = Census(census_key, year=2013)

In [3]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E", "B17001_002E", "B19301_001E","B17001_003E", "B17001_017E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "B19301_001E": "Income Per Capita",
                                      "B17001_003E": "Men Living in Poverty",
                                      "B17001_017E": "Females Living in Poverty",
                                      "zip code tabulation area": "Zipcode"})

# Add in Employment Rate (Employment Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Income Per Capita","Poverty Rate", 
                       "Poverty Count", "Men Living in Poverty", "Females Living in Poverty"]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population,Income Per Capita,Poverty Rate,Poverty Count,Men Living in Poverty,Females Living in Poverty
0,01832,22121.0,29014.0,10.903666,2412.0,1156.0,1256.0
1,01833,8295.0,43297.0,2.302592,191.0,88.0,103.0
2,01834,6675.0,40222.0,2.187266,146.0,53.0,93.0
3,01835,13527.0,31429.0,8.523693,1153.0,560.0,593.0
4,01840,4547.0,14758.0,39.542556,1798.0,696.0,1102.0


In [4]:
# Import the original data we analyzed earlier. Use dtype="object" to match other
census_data_orig = pd.read_csv("Resources/zip_bank_data.csv", dtype="object", encoding="utf-8")

# Visualize
census_data_orig.head()

,Zipcode,Address,Median Age,Household Income,Per Capita Income,Lat,Lng,Bank Count
0,624,"Hastings, MI 49058, USA",40.8,46777,22137,42.6306916,-85.2929384,9
1,692,"Ball, LA 71405, USA",35.8,55242,23941,31.4061799,-92.396174,12
2,730,"Great Mills, MD 20634, USA",31.9,79944,35961,38.2201614,-76.4967919,9
3,757,"Williamsport, TN 38487, USA",41.6,38125,18884,35.7310368,-87.2419299,0
4,957,"Marion, ND 58466, USA",44.5,69844,36981,46.5594224,-98.3481542,1


In [5]:
# Merge the two data sets along zip code
census_data_complete = pd.merge(
    census_data_orig, census_pd, how="left", on=["Zipcode", "Zipcode"])

# Remove rows missing data
census_data_complete = census_data_complete.dropna()

#Rename Columns
census_data_complete = census_data_complete.rename(columns = {'Address':'City'})

#Drop unnecessary columns
census_data_complete = census_data_complete.drop(columns=['Zipcode','Per Capita Income', 'Bank Count', 'Lat', 'Lng', 'Country'])

# Visualize
census_data_complete.head()

KeyError: "['Country'] not found in axis"

In [ ]:
#Same census date with state as the index
census_data_complete_state = census_data_complete.set_index('State')
census_data_complete_state.head()

In [ ]:
#Grab 100 random cities from Data Frame
cities_df = census_data_complete_state['City']
city_sample = random.sample(list(cities_df), 100)

#Create lists for values we want to extract
state_list = []
income_capita = []
income_household = []

#Iterate through the cities to grab data
for city in city_sample:
    sample_df=census_data_complete.loc[census_data_complete["City"] == city]
    state_list.append(sample_df.iloc[0,1])
    income_capita.append(sample_df.iloc[0,6])
    income_household.append(sample_df.iloc[0,4])
  

In [ ]:
#Use Yelp's API & create lists for city and total number of restaurants
api_key = '9nJV8mKF__qDKP8k7GzWFOoTbXjCPTCKDliNkjhv4jFyBukbefp-V1HA0gl1TYnCH3s0E0CvtwLj9440c4tlb7Wu9F94lKy9voJTqrYUQ6wSiQIzvPdYiAT69byWXXYx' 
yelp_api = YelpAPI(api_key)
total_fastfood_city = []
city_list = []

#Iterate through and append values to the lists 
for city in city_sample:
    
    search_results = yelp_api.search_query(term = 'fast food', location = city)
    total_fastfood_city.append(search_results['total'])
    city_list.append(city)
    
    
#len(total_fastfood_city)

In [ ]:
#Create DataFrame 
total_fastfood_city_df = pd.DataFrame({'State': state_list,
                                    'City': city_list,
                                    'Number of Fast Food':total_fastfood_city,
                                      'Income Per Capita':income_capita,
                                      'Household Income': income_household})

#total_fastfood_city_df.style.format({'Income Per Capita': '${:,}', 'Household Income': '${:,}'})
total_fastfood_city_df.head()

In [ ]:
#Create Bar graph of number of restaurants per city
total_fastfood_city_df_index_city=total_fastfood_city_df.set_index(['City', 'State'])
total_fastfood_city_df_index_city["Number of Fast Food"].plot(kind="bar", figsize=(20,10))
plt.xlabel("City and State")
plt.ylabel("Number of Fastfood Restaurants")
plt.title("Number of Restaurants per City")


In [ ]:
total_fastfood_city_df_index_city["Income Per Capita"].plot(kind="bar", figsize=(20,10))
# plt.xlabel("City and State")
# plt.ylabel("Number of Fastfood Restaurants")
# plt.title("Number of Restaurants per City")

In [ ]:
total_fastfood_city_df_index_city.plot(kind='scatter', x = 'Income Per Capita', y = 'Number of Fast Food', grid = True, figsize=(8,8))
plt.title("Income Per Capita vs Number of Fast Food Restaurants")
plt.show()

In [ ]:

total_fastfood_city_df_index_city["Household Income"] = total_fastfood_city_df_index_city["Household Income"].astype(int)
total_fastfood_city_df_index_city.plot(kind='scatter', x = 'Household Income', y = 'Number of Fast Food', grid = True, figsize=(8,8))
